In [88]:
import csv
from datetime import datetime
import numpy as np
import pandas as pd
import time 
from datetime import datetime
import os
from os import path
from glob import glob

import ijson
from tqdm import tqdm

## Import JSON 

In [8]:
zip_dir  = "../data/zip2"

files = glob(os.path.join(zip_dir, '*'))
for filename in files:
    if '.zip' in filename:
        pass
    elif '.json' in filename:
        pass
    else:
        newfile = filename+'.json'
        os.rename(filename, newfile)

jsons = glob(os.path.join(zip_dir, '*.json'))

## Add location data

In [10]:
dfrts = []
dfors = []
for i, json in enumerate(jsons):
    # Instance Preparation
    dates = []
    tweets = []
    user_ids = []
    
    # tweet's location
    place_fullname_tweet = []
    # tweet's country
    nationality = []
    
    start = time.time()
    # Load json file: date and tweet
    with open(json, 'r', encoding='utf8') as file:
        pet_parse = ijson.parse(file, multiple_values=True)
        for prefix, event, value  in pet_parse:
            # Date
            if prefix == 'created_at':
                dates.append(datetime.strptime(value, '%a %b %d %H:%M:%S +0000 %Y'))
            # Tweet 
            if prefix == 'text':
                tweets.append(value.replace('\n', '').replace('\t', '').replace('\r', '').replace('\r\n', '').replace('　', '')) # Delte space and indet and \r
            # User id 
            if prefix == 'user.id':
                user_ids.append(value)

            # tweet's location - fullname
            if prefix == 'place.full_name':
                place_fullname_tweet.append(value)
            if len(dates)-2 == len(place_fullname_tweet):
                place_fullname_tweet.append('nan')
                
            # tweet's nationality
            if prefix == 'place.country':
                nationality.append(value)
            if len(dates)-2 == len(nationality):
                nationality.append('nan')

        # Add missing last one data
        if len(dates) != len(place_fullname_tweet):
                place_fullname_tweet.append('nan')
        if len(dates) != len(nationality):
                nationality.append('nan')

    print("Loading json elapsed_time:{0}".format(time.time() - start) + "[sec]")
    
    # Create dataframe
    data = np.vstack([user_ids, dates, tweets, nationality, place_fullname_tweet]).T
    df = pd.DataFrame(data, columns=['user_id', 'date', 'tweet', 'nationality', 'place_tweet'])
    
    import datetime as dt
    
    # Df only with location data
    df = df[df.place_tweet != 'nan']
    
    # Change time zone
    df['date'] = df['date'] + dt.timedelta(hours = 9)
    
    # Date
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['hour'] = df['date'].dt.hour
    df['time'] = df['date'].dt.time
    df['date'] = df['date'].dt.date
    
#     # divide df by RT_flag 
#     dfrt = df[df['RT_flag'] == True]
#     dfor = df[df['RT_flag'] == False]
    
    # Save to csv file
    df.to_csv('../data/dfs_location2/df_lct'+str(i)+'.csv')

Loading json elapsed_time:718.873034954071[sec]
Loading json elapsed_time:648.138466835022[sec]
Loading json elapsed_time:526.3302919864655[sec]
Loading json elapsed_time:507.3329060077667[sec]
Loading json elapsed_time:496.4747657775879[sec]
Loading json elapsed_time:536.7752206325531[sec]
Loading json elapsed_time:555.0317189693451[sec]
Loading json elapsed_time:566.3812553882599[sec]
Loading json elapsed_time:529.7259113788605[sec]
Loading json elapsed_time:740.5443599224091[sec]


# Assign area 

In [108]:
# read csv of dictionary
def crate_area_dictionary(file):
    area = pd.read_csv(file)
    area_jp = area.drop(['pref_en'], axis=1).set_index(['pref_jp']).Area.to_dict()
    area_en = area.drop(['pref_jp'], axis=1).set_index(['pref_en']).Area.to_dict()
    return area_jp, area_en

area_jp, area_en = crate_area_dictionary('../data/area.csv')
area_shutoken_jp, area_shutoken_en = crate_area_dictionary('../data/area_shutoken.csv')
area_tokyo_jp, area_tokyo_en = crate_area_dictionary('../data/area_tokyo.csv')
area_kitakan_jp, area_kitakan_en = crate_area_dictionary('../data/area_kitakanto.csv')

# Divide tweet by area
def df_by_area(df, d_area_jp, d_area_en):
    l_area = []
    for natio, place in zip(df.nationality, df.place_tweet):
        if natio == '日本' and place.split(' ')[0] in d_area_jp:
            l_area.append(d_area_jp[place.split(' ')[0]])
        elif natio == '日本' and place.split(' ')[-1] in d_area_jp:
            l_area.append(d_area_jp[place.split(' ')[-1]])
        elif l_area == 'Japan' and place.split(', ')[-1] in d_area_en:
            l_area.append(d_area_en[place.split(', ')[-1]])
        elif natio == 'Japan' and place.split(', ')[0] in d_area_en:
            l_area.append(d_area_en[place.split(', ')[0]])
        else:
            l_area.append('nan')
    df['area'] = l_area
    df_lct = df[df.area != 'nan']
    return df_lct

In [109]:
# Read data
zip_dir  = "../data/dfs_location2" # new_dfs2_jst
csvfiles = glob(os.path.join(zip_dir, 'df*'))

dfs = []
for csvfile in csvfiles:
    df = pd.read_csv(csvfile).drop('Unnamed: 0', axis=1)
    dfs.append(df)
    
# Connet all df to one data
df_lcts = [df_by_area(df, area_jp, area_en)  for df in dfs]

## Analyze data monthly

In [111]:
# Concat dfs
for i, df in enumerate(df_lcts):
    if i == 0:
        dfm = df
    else:
        dfm = pd.concat([dfm, df], axis=0)
_df = dfm.copy()
# 首都圏
df_shutoken = df_by_area(_df, area_shutoken_jp, area_shutoken_en)
dfm = pd.concat([dfm, df_shutoken], axis=0)
# 東京
df_tokyo = df_by_area(_df, area_tokyo_jp, area_tokyo_en)
dfm = pd.concat([dfm, df_tokyo], axis=0)
# Drop axis
dfm = dfm.drop(['nationality', 'place_tweet'], axis=1)

In [112]:
# dfm

,user_id,date,tweet,year,month,day,hour,time,area
0,778167031598374912,2020-03-31,なんかもうだるい自分が嫌,2020,3,31,23,23:59:58,東北地方
1,2989602496,2020-03-31,早く!!対応してください！野党やら与党やら言ってる場合ではないですよ！亡くなった方を持ちだし...,2020,3,31,23,23:59:55,東北地方
2,986162527007490048,2020-03-31,宮藤官九郎さんが感染してしまったみたいですが22日に中山優馬くんの舞台に宮藤官九郎さんと日村...,2020,3,31,23,23:59:34,関東地方
3,3281248128,2020-03-31,はぁぁ、すでに緊張するなぁ。。。,2020,3,31,23,23:58:33,中部地方
4,50943527,2020-03-31,#志村けん面白い。見たら泣けてきた。悲しい。,2020,3,31,23,23:58:11,関東地方
...,...,...,...,...,...,...,...,...,...
4973,334036907,2020-03-01,安倍ちゃんにはまだ辞められちゃ困る。ドン底まで見届けて貰わないと,2020,3,1,0,00:13:24,東京
4975,611507926,2020-03-01,@nyantaremama ティッシュ一箱買うのに‼️何軒まわったか？結局地元のコンビニ🏪に...,2020,3,1,0,00:13:11,東京
4992,352308836,2020-03-01,株安で年金がまた、減らされる‼️株式投資しろはと言ってないのに何だか文句ばかりの、ツイート自...,2020,3,1,0,00:04:04,東京
4993,1635284976,2020-03-01,熊本は3年+2年（間あいてる）住んでたけど、パルコなくなったの悲しい。もっと長く住んでる熊本...,2020,3,1,0,00:03:22,東京


## group by area

In [113]:
area_dict = {}
# areas = dfm.area.unique().tolist()
areas = ['北海道地方',
         '東北地方',
         '関東地方',
         '中部地方',
         '近畿地方',
         '中国地方',
         '四国地方',
         '九州・沖縄地方',
         '首都圏',
         '東京']
initial_area = ['A','B','C','D','E','F','G','H','I','J']
for area, initial in zip(areas, initial_area):
    area_dict[area] = initial

In [114]:
# Save into csv of each area
for area in areas:
    dfm.groupby('area').get_group(area).to_csv('../data/dfs_by_area2/mar_'+area_dict[area]+'.csv')

In [115]:
# Read 
zip_dir  = "../data/dfs_by_area2" # new_dfs2_jst
csvfiles = glob(os.path.join(zip_dir, 'mar_*'))

feb_dfs = []
for csvfile in csvfiles:
    df = pd.read_csv(csvfile).drop(['Unnamed: 0'], axis=1)
    feb_dfs.append(df)

## Count tweet by hour

In [123]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwt_hour(dfor, filetype, savedir, key_lang, area):
    # Prepartion of dfw 
    results = []
    dfw = pd.read_csv('../data/WordTimeSeries.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'types',
                                                                                    'file name': 'file_name', 
                                                                                    'orignal form': 'orignal_form',
                                                                                    'English translation': 'English_translation'})
    booleanDictionary = {True: 'TRUE', False: 'FALSE'} # Drop nan and False in order to use query 
    dfw = dfw.replace(booleanDictionary)
    dfw = dfw.dropna(how='all')

    # Prepration of keywords
    filenames = [filename for filename in dfw.file_name.tolist() if filetype in filename]
    xd = {}
    xd['T'] = 'file_name.str.contains("T")'
    xd['D'] = 'file_name.str.contains("D")'
    xd['A'] = 'file_name.str.contains("A")'
    xd['V'] = 'file_name.str.contains("V")'
    xd['F'] = 'file_name.str.contains("F")'
    xd['C'] = 'file_name.str.contains("C")'

    query = xd[filetype]
    dfw_c = dfw.query(query, engine='python')
    
    # Keyword 
    if key_lang == 'jp':
        keywords = dfw_c.orignal_form.tolist() # English.ver: keywords = dfw_c.English_translation.tolist() 
    elif key_lang == 'en':
        keywords = dfw_c.English_translation.tolist() 
    else:
        print('Put accurate keyword language')
    
    # Find tweets
    # Extract tweet
    dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]

    for col, key in zip(filenames, keywords):
        l = []
        for row in dfor_c.itertuples():
            if key in row.tweet:
                l.append(1)
            else:
                l.append(0)
        dfor_c[col] = l

    # Save dir
    outdir = savedir
    if not os.path.exists(outdir):
            os.makedirs(outdir)

#     for i, df in enumerate(dfs):
    df = dfor_c
    unique_dates = df['date'].tolist()
    unique_dates = sorted(set(unique_dates), key=unique_dates.index) # date values
    df_date = df.groupby('date') # df grouped by date   

    for date in unique_dates:
        df_oneday = df_date.get_group(date) # df of one day
        dfh = df_oneday.groupby('hour').sum()
        dfh = dfh.drop(['year', 'month', 'user_id', 'day'],axis=1)
        
        # Save csv
        outname = 'area'+area+'_'+filetype+str(date)+'.csv'
        savename = os.path.join(outdir, outname)

        # Save to csv
        dfh.to_csv(savename)
          

In [124]:
# Run
initial_area = ['A','B','C','D','E','F','G','H','I','J']
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
for area, df in zip(initial_area, feb_dfs):
    print(area)
    for filetype in filetypes:
        findtwt_hour(df, filetype=filetype, savedir='../results_location2/hour', key_lang='jp', area=area)

A


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


B
C
D
E
F
G
H
I
J


In [127]:
# Check
for df in feb_dfs:
    print(df.area.unique())

['北海道地方']
['東北地方']
['関東地方']
['中部地方']
['近畿地方']
['中国地方']
['四国地方']
['九州・沖縄地方']
['首都圏']
['東京']
